In [0]:
%sql
USE CATALOG my_catalog

In [0]:
%sql
USE SCHEMA emp_schema

In [0]:
%sql
DESCRIBE HISTORY employees

##Delta timetravel
### In Delta Lake we can easily query previous versions of our table, and this feature of time travel


In [0]:
%sql
/* So let us say we want to access our data before the update operation.*/
--COMMAND: BY using Version AS OF keyword by specifyiong the version number --
select * from employees version as of 6;



   

In [0]:
%sql
--Another alternative syntax for timetravel is select statement with@v followed by version number--
select * from employees@v6


In [0]:
%sql


In [0]:
%sql
/*Now imagine this scenario,

we deleted our data and we need to restore them.

Here, all data has been removed.*/
Delete from employees;

In [0]:
%sql
--lets conform whesther data is removed or not by running the select command--
select * from employees;

In [0]:
%sql
--now there are no rows in our table--
/*We can simply roll back to our previous version

before deletion using the Restore Table command.*/
RESTORE TABLE EMPLOYEES TO VERSION AS OF 8;

In [0]:
%sql
--lets confirm wheather the files are restpred or not--
select * from employees;

In [0]:
%sql
--let us know what happened in our table--
describe history employees;

##Optimize command

Having many small data files

negatively affect the performance of the delta table.

To resolve this issue, we can use Optimize command

that combined files toward an optimal size.

Optimize will replace existing data files

by combining records and rewriting the results.


In [0]:
%sql
describe detail employees

In [0]:
%sql
--Command optimse tablename--
/* Here, we can see that our six data files

have been soft deleted.

And a new file has been added that compact those six files.

In addition, you may noticed that we added

the Z order indexing with our Optimize command.

Z order indexing speeds up data retrieval

when filtering on provided fields by grouping data

with similar values within the same data files.

In our case, we do Z order by the ID column.

However, on such a small data set,

it does not provide any benefit.*/
OPTIMIZE employees zorder by id

In [0]:
%sql
/*Let us confirm the output

of the Optimize command by running Describe Detail

on our table.*/
describe detail employees;
/*
Here, we can see that the number of files

in the current version is only one.*/

In [0]:
%sql
--Let us see how the Optimize operation has been recorded in our table history.--
describe history employees
--As expected, Optimize command created another version in our table.--

In [0]:
%sql
--vacuum command :We can manually remove all data files using the Vacuum command.--
vacuum employees;
--after this check file directory %fs ls "directory path" nothing will happend beacuase we did not specify the retention period"--
/* by  default retention period id 7 days That means that Vacuum operation will prevent us

from deleting files less than seven days old,

just to ensure that no longer running operation

are still referencing any of the files to be deleted.*/

In [0]:
%sql
-- after running the below command check the file directoty %fs ls path --
vacuum employees retain 0 hours


In [0]:
%sql
SET spark.databricks.delta.retentionDurationCheck.enabled = false;
--NOW WE CAN RUN VACCUM COMMAND--
-- after running the below command check the file directoty %fs ls path --
vacuum employees retain 0 hour
--after running the vacuum command check the table table directory here some files are deleted--
/*So now we are no longer able to access old data versions.

We can easily confirm this by querying an old table version.*/
select * from employees @v1
--if we run the above command we will get file not found exception because the data files for this version no longer exist.--

In [0]:
%sql
--Finnaly drop the table--
drop table employees


In [0]:
%sql
select * from employees